### Setting up

In [57]:
import backtrader as bt
import pandas as pd
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
from collections import deque
from tabulate import tabulate
st_date = datetime.datetime(2022, 5, 23)
ed_date = datetime.datetime(2022, 6, 12)

In [58]:

if_codes = ['IF1910','IF1911','IF1912',
            'IF2001','IF2002','IF2003','IF2004','IF2005','IF2006',
            'IF2007','IF2008','IF2009','IF2010','IF2011','IF2012',
            'IF2101','IF2102','IF2103','IF2104','IF2105','IF2106',
            'IF2107','IF2108','IF2109','IF2110','IF2111','IF2112',
            'IF2201','IF2202','IF2203','IF2204','IF2205','IF2206']
ic_codes = ['IC1910','IC1911','IC1912',
            'IC2001','IC2002','IC2003','IC2004','IC2005','IC2006',
            'IC2007','IC2008','IC2009','IC2010','IC2011','IC2012',
            'IC2101','IC2102','IC2103','IC2104','IC2105','IC2106',
            'IC2107','IC2108','IC2109','IC2110','IC2111','IC2112',
            'IC2201','IC2202','IC2203','IC2204','IC2205','IC2206']
ih_codes = ['IH1910','IH1911','IH1912',
            'IH2001','IH2002','IH2003','IH2004','IH2005','IH2006',
            'IH2007','IH2008','IH2009','IH2010','IH2011','IH2012',
            'IH2101','IH2102','IH2103','IH2104','IH2105','IH2106',
            'IH2107','IH2108','IH2109','IH2110','IH2111','IH2112',
            'IH2201','IH2202','IH2203','IH2204','IH2205','IH2206']
cn_codes =  [
            'CN2111','CN2112','CN2201','CN2202',
            'CN2203','CN2204','CN2205','CN2206'
            ]
if_data = {}
ic_data = {}
ih_data = {}
cn_data = {}


for a in if_codes:
    if_data[a] =  pd.read_csv('Data/IF/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True).dropna()
for a in ic_codes:
    ic_data[a] =  pd.read_csv('Data/IC/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True).dropna()
for a in ih_codes:
    ih_data[a] =  pd.read_csv('Data/IH/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True).dropna()
for a in cn_codes:
    cn_data[a] =  pd.read_csv('Data/CN/'+a+'.csv', index_col = 0,skiprows = [0,1,3],parse_dates=True).dropna()

csi300=pd.read_csv('Data\CSI300.csv', index_col = 0,skiprows = [0],parse_dates=True)
csi500= pd.read_csv('Data\CSI500.csv',index_col = 0,skiprows = [0],parse_dates=True)
sse50= pd.read_csv('Data\SSE50.csv',index_col = 0,skiprows = [0],parse_dates=True)
a50= pd.read_csv('Data\A50.csv',index_col = 0,skiprows = [0,1],parse_dates=True)


In [59]:
if_feeds = []
ic_feeds = []
cn_feeds = []
ih_feeds = []

if_rolldate = []
ic_rolldate = []
ih_rolldate = []

cn_rolldate = []

for k in if_data.keys():
    if if_data[k].index[-1] < pd.to_datetime(st_date):
        # print(if_data[k].index[-1])
        continue
    if_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    if_data[k]['Volume'] = if_data[k]['Volume']*100000
    if_data[k] = if_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    if_data[k] = if_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    if_feeds.append(bt.feeds.PandasData(dataname = if_data[k],name = k))
    if_rolldate.append(if_data[k].resample('1d').last().index[-2]+datetime.timedelta(hours=14,minutes=57))

for k in ic_data.keys():
    if ic_data[k].index[-1] < pd.to_datetime(st_date):
        # print(ic_data[k].index[-1])
        continue
    ic_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    ic_data[k]['Volume'] = ic_data[k]['Volume']*100000
    ic_data[k] = ic_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    ic_data[k] = ic_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    ic_feeds.append( bt.feeds.PandasData(dataname = ic_data[k],name = k))

for k in ih_data.keys():
    if ih_data[k].index[-1] < pd.to_datetime(st_date):
        # print(ih_data[k].index[-1])
        continue
    ih_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    ih_data[k]['Volume'] = ih_data[k]['Volume']*100000
    ih_data[k] = ih_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    ih_data[k] = ih_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    ih_feeds.append( bt.feeds.PandasData(dataname = ih_data[k],name = k))



csi300 = csi300.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
csi300.fillna(inplace = True,method = 'ffill')
csi500 = csi500.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
csi500.fillna(inplace = True,method = 'ffill')
sse50 = sse50.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
sse50.fillna(inplace = True,method = 'ffill')
a50 = a50.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
a50.fillna(inplace = True,method = 'ffill')

for k in cn_data.keys():    
    # cn_data[k]
    if cn_data[k].index[-1] < pd.to_datetime(st_date):
        # print(cn_data[k].index[-1])
        continue

    cn_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    cn_data[k]['Volume'] = cn_data[k]['Volume']*100000
    cn_data[k] = cn_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    cn_data[k] = cn_data[k].fillna(method= 'ffill')
    cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
    cn_feeds.append(bt.feeds.PandasData(dataname = cn_data[k],name = k))
    cn_rolldate.append(cn_data[k].resample('1d').last().index[-2]+datetime.timedelta(hours=14,minutes=57))

csi300_feed = bt.feeds.PandasData(dataname = csi300,fromdate=st_date,todate=ed_date,)
csi500_feed = bt.feeds.PandasData(dataname = csi500,fromdate=st_date,todate=ed_date,)
sse50_feed = bt.feeds.PandasData(dataname = sse50,fromdate=st_date,todate=ed_date,)

a50_feed = bt.feeds.PandasData(dataname = a50,fromdate=st_date,todate=ed_date,)
if_rolldate=if_rolldate[:-1]
# ic_rolldate = if_rolldate
cn_rolldate = cn_rolldate[:-1]

def cn_expire(dt,d):
    return dt in cn_rolldate

def if_expire(dt,d):
    if dt in if_rolldate :
        return True
    return False



In [60]:
class IFICStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',27),
        ('logname','log.csv'),

    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close[0],
                                            self.csi500_close[0],
                                            self.a50_close[0],
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    

    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                       
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Datetime,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = self.data3.close
        self.csi500_close = self.data4.close
        self.a50_close = self.data5.close
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        
    def next(self):
        
        self.cn_theo = self.a50_close[0]*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*self.csi300_close[0]-self.p.ic_lot*2*self.csi500_close[0])
        print(self.a50_close[0],self.csi300_close[0],self.csi500_close[0],self.if_close[0],self.ic_close[0])
        cn_bid = self.cn_theo - self.p.margin + self.base
        cn_ask = self.cn_theo + self.p.margin + self.base
        self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
        self.write_record()
        if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
            size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
            self.buy( data = self.getdatabyname('IF'),
                      size = self.p.if_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('IC'),
                       size = self.p.ic_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('CN'),
                       size = self.p.cn_lot*size, # size < 0
                       exectype=bt.Order.Market,coc=True)
            self.base = self.base + size * self.p.distor # base 上移，抬高理论价
            
        if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
            size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
            self.sell( data = self.getdatabyname('IF'),
                       size = self.p.if_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('IC'),
                      size = self.p.ic_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('CN'),
                      size = self.p.cn_lot*size, # size < 0
                      exectype=bt.Order.Market,coc=True)
            self.base = self.base - size * self.p.distor

        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
            print('ROLL')
            self.base = 0

            if self.getpositionbyname('IF').size != 0:
                self.order_target_size( data = self.getdatabyname('IF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('IC').size != 0:
                self.order_target_size( data = self.getdatabyname('IC'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
            print('ROLL')
            self.base = 0
            if self.getpositionbyname('IF').size != 0:
                self.order_target_size( data = self.getdatabyname('IF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('IC').size != 0:
                self.order_target_size( data = self.getdatabyname('IC'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)

    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [61]:
class IFICAbsDailyOpenStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            # if self.datetime.time() == datetime.time(hour = 9, minute=30):
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.datetime.time() == datetime.time(hour = 9, minute=30): #and self.datetime.date().isocalendar()[1] >= self.week+4:
                self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*self.csi300_close-self.p.ic_lot*2*self.csi500_close)
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [62]:
class IFICAbsDailyCloseStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            # if self.datetime.time() == datetime.time(hour = 9, minute=30):
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.datetime.time() == datetime.time(hour = 14, minute=59): #and self.datetime.date().isocalendar()[1] >= self.week+4:
                print('REbasing!')
                self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*self.csi300_close-self.p.ic_lot*2*self.csi500_close)
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [63]:
class IFICAbsRollStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            if self.datetime.time() == datetime.time(hour = 9, minute=30):
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.data.datetime.datetime() in if_rolldate:# or self.data.datetime.datetime() in cn_rolldate:#and self.datetime.date().isocalendar()[1] >= self.week+4:
                # self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*self.csi300_close-self.p.ic_lot*2*self.csi500_close)
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    # def next_open(self):
        # 换月重新开仓
    #     # if self.data.datetime.datetime()in if_rolldate:  
    #             self.buy(data=self.getdatabyname('IF'),exectype=bt.Order.Market,coo = True)
    #             self.buy(data=self.getdatabyname('IC'),exectype=bt.Order.Market,coo = True)
    #     if self.data.datetime.datetime()in cn_rolldate:  
    #             self.buy(data=self.getdatabyname('CN'),exectype=bt.Order.Market,coo = True)
    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [ ]:
class IFICReplayStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            if self.datetime.time() == datetime.time(hour = 9, minute=30):
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.data.datetime.datetime() in if_rolldate:# or self.data.datetime.datetime() in cn_rolldate:#and self.datetime.date().isocalendar()[1] >= self.week+4:
                # self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*self.csi300_close-self.p.ic_lot*2*self.csi500_close)
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    # def next_open(self):
        # 换月重新开仓
    #     # if self.data.datetime.datetime()in if_rolldate:  
    #             self.buy(data=self.getdatabyname('IF'),exectype=bt.Order.Market,coo = True)
    #             self.buy(data=self.getdatabyname('IC'),exectype=bt.Order.Market,coo = True)
    #     if self.data.datetime.datetime()in cn_rolldate:  
    #             self.buy(data=self.getdatabyname('CN'),exectype=bt.Order.Market,coo = True)
    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [ ]:
from tabulate import tabulate

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(100000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='IF', *if_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='IC', *ic_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date,)
cerebro.rolloverdata(name='CN', *cn_feeds,checkdate = cn_expire,fromdate=st_date,todate=ed_date,)
cerebro.adddata(csi300_feed)
cerebro.adddata(csi500_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=300,
                             margin=150000,
                             name='IF')
cerebro.broker.setcommission(commission=27,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=200,
                             margin=160000,
                             name='IC')
cerebro.broker.setcommission(commission=10.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=6.8,
                             margin=10000,
                             name='CN')
cerebro.addstrategy(IFICReplayStrategy,if_lot = 0.000001,cn_lot = 0.000001,ic_lot = 0.000001,logname = 'replay_523-610.csv')
cerebro.add_order_history(tradelist, notify=True)

# cerebro.addstrategy(IFICAbsRollStrategy,if_lot = 2,cn_lot = 14,logname = '2IF_IC_27CN_Roll_523-610.csv')
strats = cerebro.run(runonce = False,cheat_on_open=True,tradehistory = True)


In [73]:
port_record_set = {}
port_record = pd.read_csv('portlog/2IF_IC_27CN_ROLL_523-610.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Roll'] = port_record


# port_record = pd.read_csv('portlog/3IF_IC_27CN_ABS_ROLLN_JUN_10.csv')
# port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
# port_record.set_index('Datetime',inplace=True)
# port_record = port_record.dropna()
# port_record.index = pd.to_datetime(port_record.index)

# port_record_set['RollN'] = port_record


# abs_port_record = pd.read_csv('portlog/3IF_IC_27CN_ABS_DailyC_JUN_10.csv')
# abs_port_record = abs_port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
# abs_port_record.set_index('Datetime',inplace=True)
# abs_port_record = abs_port_record.dropna()
# abs_port_record.index = pd.to_datetime(abs_port_record.index)

# port_record_set['DailyClose'] = abs_port_record

port_record = pd.read_csv('portlog/2IF_IC_27CN_DailyO_523-610.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['DailyOpen'] = port_record



port_record = pd.read_csv('portlog/2IF_IC_27CN_Basis_523-610.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Basis'] = port_record


In [41]:
for k in port_record_set.keys():
    port_record_set[k] = port_record_set[k].loc['2022-06-01':]

In [107]:
tf = pd.read_csv('tradeinfo.csv',parse_dates = True,infer_datetime_format=True,header=None)

In [108]:
tf.columns = ['datetime','dir','ct','price','lot','dd','tp','info','g','ddd']
tf = tf.set_index('datetime')


In [110]:
tf.index = pd.to_datetime(tf.index,format='%Y-%m-%d_%H:%M:%S.%f')

In [116]:
tf

,dir,ct,price,lot,dd,tp,info,g,ddd
datetime,,,,,,,,,
2022-05-23 09:30:12.868,sell,SGX_CN_2205,13530.0,6,dir,2,info=,13080.30,NaN
2022-05-23 09:30:12.920,sell,SGX_CN_2205,13530.0,21,dir,2,info=,13080.30,NaN
2022-05-23 09:30:12.941,buy,IF2206,4054.8,1,dir,1,info=,3938.76,4054.8
2022-05-23 09:30:12.942,buy,IF2206,4055.0,1,dir,1,info=,3938.76,4054.8
2022-05-23 09:30:12.942,buy,IF2206,4055.0,1,dir,1,info=,3938.76,4054.8
...,...,...,...,...,...,...,...,...,...
2022-06-10 14:45:50.283,buy,SGX_CN_2206,14079.0,16,dir,1,info=,14103.60,NaN
2022-06-10 14:45:50.305,buy,SGX_CN_2206,14079.0,11,dir,1,info=,14103.60,NaN
2022-06-10 14:45:50.309,sell,IF2206,4242.8,1,dir,2,info=,4239.43,4242.8


In [128]:
tradelist

[(Timestamp('2022-05-23 09:30:12.868000'), -6, 13530.0, 'CN'),
 (Timestamp('2022-05-23 09:30:12.920000'), -21, 13530.0, 'CN'),
 (Timestamp('2022-05-23 09:30:12.941000'), -1, 4054.8, 'IF'),
 (Timestamp('2022-05-23 09:30:12.942000'), -1, 4055.0, 'IF'),
 (Timestamp('2022-05-23 09:30:12.942000'), -1, 4055.0, 'IF'),
 (Timestamp('2022-05-23 09:30:12.943000'), -1, 5878.6, 'IC'),
 (Timestamp('2022-05-23 09:30:47.010000'), -21, 13517.0, 'CN'),
 (Timestamp('2022-05-23 09:30:47.011000'), -2, 13517.0, 'CN'),
 (Timestamp('2022-05-23 09:30:47.023000'), -1, 4050.4, 'IF'),
 (Timestamp('2022-05-23 09:30:47.023000'), -1, 4050.6, 'IF'),
 (Timestamp('2022-05-23 09:30:47.030000'), -4, 13517.0, 'CN'),
 (Timestamp('2022-05-23 09:30:47.054000'), -1, 4050.6, 'IF'),
 (Timestamp('2022-05-23 09:30:47.055000'), -1, 5876.4, 'IC'),
 (Timestamp('2022-05-23 09:31:19.718000'), -27, 13489.0, 'CN'),
 (Timestamp('2022-05-23 09:31:19.738000'), -2, 4043.8, 'IF'),
 (Timestamp('2022-05-23 09:31:19.739000'), -1, 4043.8, 'IF'),

In [127]:
# tf.index = pd.to_datetime(tf.index)
tf = tf.loc['2022-5-23':'2022-6-12']
tradelist = []
for idx,r in tf.iterrows():
    print(r)
    ct = 'CN'
    if 'IF' in r['ct']:
        ct = 'IF'
    if 'IC' in r['ct']:
        ct = 'IC'
    size = r['lot'] if r['dir'] is 'buy' else -r['lot']
    tradelist.append(
                        (
                        idx,#datetime.datetime.strftime(idx,'%Y-%m-%d %H:%M:%S'),
                        size,
                        r['price'],
                        ct
                        )
                    )

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Windows\Temp\ipykernel_20076\1860709297.py:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  size = r['lot'] if r['dir'] is 'buy' else -r['lot']


dir             sell
ct       SGX_CN_2205
price          13530
lot                6
dd               dir
tp                 2
info           info=
g            13080.3
ddd              NaN
Name: 2022-05-23 09:30:12.868000, dtype: object
dir             sell
ct       SGX_CN_2205
price          13530
lot               21
dd               dir
tp                 2
info           info=
g            13080.3
ddd              NaN
Name: 2022-05-23 09:30:12.920000, dtype: object
dir          buy
ct        IF2206
price     4054.8
lot            1
dd           dir
tp             1
info       info=
g        3938.76
ddd       4054.8
Name: 2022-05-23 09:30:12.941000, dtype: object
dir          buy
ct        IF2206
price       4055
lot            1
dd           dir
tp             1
info       info=
g        3938.76
ddd       4054.8
Name: 2022-05-23 09:30:12.942000, dtype: object
dir          buy
ct        IF2206
price       4055
lot            1
dd           dir
tp             1
info       info=
g    

In [74]:
port_record = port_record.resample('5min').last().dropna()
totalPortValue = (
        Line(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis([a.isoformat(sep=' ') for a in port_record_set['Basis'].index.to_list()])
        .add_yaxis(
                    series_name="基差",
                    y_axis=round(port_record_set['Basis'].Value/1000).values.tolist()-port_record_set['Basis'].Value[0]/1000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width =1)
                    )
        .add_yaxis(
                    series_name="绝对值_DailyO",
                    y_axis=round(port_record_set['DailyOpen'].Value/1000).values.tolist()-port_record_set['DailyOpen'].Value[0]/1000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width = 1)
                    
                    )

        .add_yaxis(
                    series_name="绝对值_Roll",
                    y_axis=round(port_record_set['Roll'].Value/1000).values.tolist()-port_record_set['Roll'].Value[0]/1000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width = 1)
        )
        # .add_yaxis(
        #             series_name="绝对值_RollN",
        #             y_axis=round(port_record_set['RollN'].Value/1000).values.tolist()-port_record_set['RollN'].Value[0]/1000,
        #             is_symbol_show = False,
        #             label_opts=opts.LabelOpts(is_show=False),
        #             linestyle_opts=opts.LineStyleOpts(width = 1)
        # # )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'CN Pos',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                    )
        
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            title_opts = opts.TitleOpts('3IF-IC-27CN',),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Value',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                                # opts.DataZoomOpts(
                                #     is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                                # ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )
totalCashUsage = (
    Bar()
    .add_xaxis([a.isoformat(sep=' ') for a in port_record.index.to_list()])
    .add_yaxis(
                series_name="基差",
                y_axis=port_record_set['Basis'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.05, )
    )
    .add_yaxis(
                series_name="绝对值_Roll",
                y_axis=port_record_set['Roll'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.05, )
    )
    .add_yaxis(
                series_name="绝对值_DailyO",
                y_axis=port_record_set['DailyOpen'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.05, )
    )

   
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'position',
                                # max_ = 200,
                                # min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),

    )
)

totalPortValue.overlap(totalCashUsage)
totalPortValue.render('3ific.html')


'c:\\Users\\Alienware\\Desktop\\ah\\股指\\3ific.html'

In [294]:
class Trade(object): 
	def __init__(self, time, symbol, dir, price, size, comm): 
		self.time = time 
		self.symbol = symbol
		self.dir = dir 
		self.price = price 
		self.size = size
		self.comm = comm
	def __str__(self):
		return "{}, {}, {}, {}, {}, {}".format(self.time, self.symbol, self.dir, self.size, self.price,self.comm)
class ClosedTrade(object):
	def __init__(self,open_trade,close_trade):
		self.open_time = open_trade.time
		self.close_time = close_trade.time
		self.symbol = close_trade.size
		self.size = close_trade.size
		self.pnl = pnl
		self.open_side = open_trade.dir
		self.close_side = close_trade.dir
		self.open_price = open_trade.price
		self.close_price = close_trade.price

	# For printing out the closed trades
	def __str__(self):
		return "{},{},{},{},{},{},{},{},{}".format(self.open_time, self.close_time, self.symbol, self.quantity, self.pnl, self.open_side, self.close_side, self.open_price, self.close_price)


In [295]:
class p_2IFICCN(object): 
	def __init__(self,log_if,log_ic,log_cn,ref,margin): 
		self.time = log_if.Datetime 
		self.value = log_cn.Price * 15 * 6.5 - log_if.Price * 2 * 300 + log_ic.Price * 200 * 1
		self.comm = log_ic.Comm+log_cn.Comm+log_if.Comm
		self.dir = log_cn.Trade
		self.ref = ref
		self.basis_profit = 15*margin*6.5
	def __str__(self):
		return "{}, {}, {}, {}, {}".format(self.time, self.dir, self.value,self.comm,self.ref)

class p_3IFICCN(object): 
	def __init__(self,log_if,log_ic,log_cn,ref,margin): 
		self.time = log_if.Datetime 
		self.value = log_cn.Price * 30 * 6.5 - log_if.Price * 3 * 300 + log_ic.Price * 200 * 1
		self.comm = log_ic.Comm+log_cn.Comm+log_if.Comm
		self.dir = log_cn.Trade
		self.ref = ref
		self.basis_profit = 30*margin*6.5

	def __str__(self):
		return "{}, {}, {}, {}, {}".format(self.time, self.dir, self.value,self.comm,self.ref)

class ClosedPair(object):
	def __init__(self,open,close):
		self.open_time = open.time
		self.close_time = close.time
		self.comm = open.comm+close.comm
		self.pnl = ((close.value - open.value) if open.dir == 'Buy' else -(close.value - open.value)) - self.comm
		self.basis_profit = close.basis_profit
		self.dir = open.dir
		self.open_ref = open.ref
		self.clsoe_ref = close.ref
	def __str__(self):
		return "{}, {}, {}, {}".format(self.open_time, self.close_time, self.dir,self.pnl)

In [296]:
trade_list = []
for idx,row in trade_log.iterrows():
	trade_list.append(Trade(idx,row.Symbol,row.Trade,row.Price,row.Size,row.Comm))
temp_trade_log = trade_log.reset_index()

abs_trade_list = []
for idx,row in abs_trade_log.iterrows():
	abs_trade_list.append(Trade(idx,row.Symbol,row.Trade,row.Price,row.Size,row.Comm))
abs_temp_trade_log = abs_trade_log.reset_index()

In [ ]:
pair_list = []
count = 0
for i in range(0,len(temp_trade_log)//3):
	if_row = temp_trade_log.iloc[3*i]
	ic_row = temp_trade_log.iloc[3*i+1]
	cn_row = temp_trade_log.iloc[3*i+2]
	n = ic_row.Size
	for d in range(0,abs(n)):
		pair_list.append(p_3IFICCN(if_row,ic_row,cn_row,count,35))
		count = count + 1

abs_pair_list = []
count = 0
for i in range(0,len(abs_temp_trade_log)//3):
	if_row = abs_temp_trade_log.iloc[3*i]
	ic_row = abs_temp_trade_log.iloc[3*i+1]
	cn_row = abs_temp_trade_log.iloc[3*i+2]
	n = ic_row.Size
	for d in range(0,abs(n)):
		abs_pair_list.append(p_3IFICCN(if_row,ic_row,cn_row,count,35))
		count = count + 1

In [298]:
closed_list = []
position_stack = deque()
port_record['Win'] = np.nan
port_record['Lose'] = np.nan

for item in pair_list:
    if len(position_stack) == 0 or item.dir == position_stack[-1].dir:
        position_stack.append(item)
    else:
        closedItem = ClosedPair(position_stack.pop(),item)
        closed_list.append(closedItem)
        if closedItem.pnl > 0:
            port_record.loc[closedItem.close_time,'Win'] = closedItem.pnl
        else:
            port_record.loc[closedItem.close_time,'Lose'] = closedItem.pnl




In [ ]:
abs_closed_list = []
abs_position_stack = deque()
abs_port_record['PnL'] = np.nan
for item in abs_pair_list:
    if len(abs_position_stack) == 0 or item.dir == abs_position_stack[-1].dir:
        abs_position_stack.append(item)
    else:
        closedItem = ClosedPair(abs_position_stack.pop(),item)
        abs_closed_list.append(closedItem)
        if closedItem.pnl > 0:
            abs_port_record.loc[closedItem.close_time,'Win'] = closedItem.pnl
        else:
            abs_port_record.loc[closedItem.close_time,'Lose'] = closedItem.pnl


In [303]:
def tradeMark(trade_date,trade_dir,trade_price):
    if trade_dir == 'Sell':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'circle',
                                    symbol_size = 10,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'green') )
    if trade_dir == 'Buy':
        return opts.MarkPointItem(  coord=[trade_date.date(),trade_price],
                                    symbol = 'circle',
                                    symbol_size = 10,
                                    itemstyle_opts = opts.ItemStyleOpts(color = 'red') )

### Integrating into Grid